In [53]:
import numpy as np
import pandas as pd
import math

In [54]:
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,Normalizer,MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor

In [55]:
df1=pd.read_csv("train.csv")
df2=pd.read_csv("meal_info.csv")
df3=pd.read_csv("fulfilment_center_info.csv")

In [56]:
df1 = df1.sample(frac=1).reset_index(drop=True)

In [57]:
df1.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1001927,71,94,1754,191.12,292.03,1,0,865
1,1213213,130,109,1971,323.04,323.04,0,0,109
2,1370387,2,92,1885,98.03,114.46,1,1,595
3,1077621,54,92,1543,475.33,476.33,0,0,40
4,1146235,15,104,2290,302.70,300.70,0,0,487


In [58]:
df2.head()

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian


In [59]:
df3.head()

,center_id,city_code,region_code,center_type,op_area
0,11,679,56,TYPE_A,3.7
1,13,590,56,TYPE_B,6.7
2,124,590,56,TYPE_C,4.0
3,66,648,34,TYPE_A,4.1
4,94,632,34,TYPE_C,3.6


In [60]:
df1.info()
df1.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456548 entries, 0 to 456547
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     456548 non-null  int64  
 1   week                   456548 non-null  int64  
 2   center_id              456548 non-null  int64  
 3   meal_id                456548 non-null  int64  
 4   checkout_price         456548 non-null  float64
 5   base_price             456548 non-null  float64
 6   emailer_for_promotion  456548 non-null  int64  
 7   homepage_featured      456548 non-null  int64  
 8   num_orders             456548 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 31.3 MB


Index(['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'num_orders'],
      dtype='object')

In [61]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   meal_id   51 non-null     int64 
 1   category  51 non-null     object
 2   cuisine   51 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.3+ KB


In [62]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   center_id    77 non-null     int64  
 1   city_code    77 non-null     int64  
 2   region_code  77 non-null     int64  
 3   center_type  77 non-null     object 
 4   op_area      77 non-null     float64
dtypes: float64(1), int64(3), object(1)
memory usage: 3.1+ KB


In [63]:
x=df1[['id', 'week', 'center_id', 'meal_id', 'checkout_price', 
    'base_price','emailer_for_promotion', 'homepage_featured']]
y=df1['num_orders']

In [64]:
x['center_id'].nunique()

77

In [65]:
x,xt,y,yt=train_test_split(x,y,test_size=0.9,random_state=0)

In [66]:
x.shape

(45654, 8)

In [67]:
sc=StandardScaler()
x=sc.fit_transform(x)

In [68]:
# feature extraction
model=RandomForestRegressor()
pca=PCA(n_components=4)
xbest=pca.fit_transform(x)

In [69]:
xtrain,xtest,ytrain,ytest=train_test_split(xbest,y,test_size=0.2,random_state=0)
model.fit(xtrain,ytrain)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [70]:
predict=model.predict(xtest)

In [71]:
ans=pd.DataFrame(predict.reshape(-1,1),columns=['predict'])

In [72]:
ans['ori']=ytest.values.reshape(-1,1)
ans['error_%'] = abs(((ans['predict']-ans['ori'])/ans['ori'])*100)

In [73]:
ans[ans['error_%']<50].count()

predict    3900
ori        3900
error_%    3900
dtype: int64

In [74]:
ans['error_%'].mean()

188.64596642836014

In [75]:
#saving the model
import pickle
with open('food_model.pkl','wb') as file:
    pickle.dump(model,file)

In [76]:
#saving scaler
with open('food_scale.pkl','wb') as file:
    pickle.dump(sc,file)                   

In [77]:
#saving pca
with open('food_col.pkl','wb') as file:
    pickle.dump(pca,file)